In [1]:

import time 
import sys
import numpy as np 



from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw


import pandas as pd
import argparse

import requests
import os
import codecs

from pandas.io.html import read_html

!pip install wikipedia
!pip install lxml
!pip install html5lib
!pip install BeautifulSoup4


     |████████████████████████████████| 102kB 10.9MB/s ta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia
     |████████████████████████████████| 5.8MB 1.9MB/s eta 0:00:01


In [3]:
import wikipedia as wp


from pandas.io.html import read_html

from lxml import etree as et

In [4]:

page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

wikitables = read_html(page,  attrs={"class":"wikitable"})

print ("Extracted {num} wikitables".format(num=len(wikitables)))

Extracted 1 wikitables


In [5]:

wikitables[0].head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [6]:
wikitables

[    Postcode           Borough          Neighbourhood
 0        M1A      Not assigned           Not assigned
 1        M2A      Not assigned           Not assigned
 2        M3A        North York              Parkwoods
 3        M4A        North York       Victoria Village
 4        M5A  Downtown Toronto           Harbourfront
 ..       ...               ...                    ...
 283      M8Z         Etobicoke              Mimico NW
 284      M8Z         Etobicoke     The Queensway West
 285      M8Z         Etobicoke  Royal York South West
 286      M8Z         Etobicoke         South of Bloor
 287      M9Z      Not assigned           Not assigned
 
 [288 rows x 3 columns]]

In [7]:
type(wikitables)

list

#####The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [8]:
wiki1 = pd.DataFrame(wikitables[0], columns =['Postcode', 'Borough', 'Neighbourhood'])

In [9]:
wiki1

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
283,M8Z,Etobicoke,Mimico NW
284,M8Z,Etobicoke,The Queensway West
285,M8Z,Etobicoke,Royal York South West
286,M8Z,Etobicoke,South of Bloor


In [10]:
type(wiki1)
wiki1.shape

(288, 3)

In [11]:
wiki3=wiki1

####If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [12]:
wiki3.loc[wiki3.Neighbourhood == 'Not assigned', 'Neighbourhood'] = wiki3['Borough']

wiki3.loc[wiki3.Neighbourhood != 'Not assigned', 'Neighbourhood'] = wiki3['Neighbourhood']

In [13]:
wiki3

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
283,M8Z,Etobicoke,Mimico NW
284,M8Z,Etobicoke,The Queensway West
285,M8Z,Etobicoke,Royal York South West
286,M8Z,Etobicoke,South of Bloor


In [14]:
wiki3.iloc[:9]

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park


In [15]:
wiki4=wiki3

#####hese two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 

In [16]:
wiki4.set_index(['Postcode','Borough'],inplace=True)

In [17]:
result3 = wiki4.groupby(level=['Postcode','Borough'], sort=False).agg( ','.join)

In [18]:
result3

,,Neighbourhood
Postcode,Borough,
M1A,Not assigned,Not assigned
M2A,Not assigned,Not assigned
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront,Regent Park"
...,...,...
M5Z,Not assigned,Not assigned
M6Z,Not assigned,Not assigned
M7Z,Not assigned,Not assigned


In [19]:
result3[1:10]

,,Neighbourhood
Postcode,Borough,
M2A,Not assigned,Not assigned
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront,Regent Park"
M6A,North York,"Lawrence Heights,Lawrence Manor"
M7A,Queen's Park,Queen's Park
M8A,Not assigned,Not assigned
M9A,Etobicoke,Islington Avenue
M1B,Scarborough,"Rouge,Malvern"


In [20]:
result3.shape

(180, 1)

In [22]:
results4=result3

#####Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned

In [34]:
results4[results4.Neighbourhood != 'Not assigned']

,,Neighbourhood
Postcode,Borough,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront,Regent Park"
M6A,North York,"Lawrence Heights,Lawrence Manor"
M7A,Queen's Park,Queen's Park
...,...,...
M8X,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North"
M4Y,Downtown Toronto,Church and Wellesley
M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern


####In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe

In [36]:
results4.shape

(180, 1)

#####question 2 

In [25]:
coordinates = pd.read_csv("Geospatial_Coordinates.csv") 

In [27]:
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [37]:
results5=results4
results5

,,Neighbourhood
Postcode,Borough,
M1A,Not assigned,Not assigned
M2A,Not assigned,Not assigned
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront,Regent Park"
...,...,...
M5Z,Not assigned,Not assigned
M6Z,Not assigned,Not assigned
M7Z,Not assigned,Not assigned


In [38]:
results5 = pd.merge(results5 , coordinates, 
        left_on='Postcode', right_on='Postal Code')

In [39]:
results5

,Neighbourhood,Postal Code,Latitude,Longitude
0,Parkwoods,M3A,43.753259,-79.329656
1,Victoria Village,M4A,43.725882,-79.315572
2,"Harbourfront,Regent Park",M5A,43.654260,-79.360636
3,"Lawrence Heights,Lawrence Manor",M6A,43.718518,-79.464763
4,Queen's Park,M7A,43.662301,-79.389494
...,...,...,...,...
98,"The Kingsway,Montgomery Road,Old Mill North",M8X,43.653654,-79.506944
99,Church and Wellesley,M4Y,43.665860,-79.383160
100,Business Reply Mail Processing Centre 969 Eastern,M7Y,43.662744,-79.321558
101,"Humber Bay,King's Mill Park,Kingsway Park Sout...",M8Y,43.636258,-79.498509


In [45]:
results7 = pd.merge(results4 , results5, 
        left_on='Neighbourhood', right_on='Neighbourhood')

In [46]:
results7

,Neighbourhood,Postal Code,Latitude,Longitude
0,Parkwoods,M3A,43.753259,-79.329656
1,Victoria Village,M4A,43.725882,-79.315572
2,"Harbourfront,Regent Park",M5A,43.654260,-79.360636
3,"Lawrence Heights,Lawrence Manor",M6A,43.718518,-79.464763
4,Queen's Park,M7A,43.662301,-79.389494
...,...,...,...,...
98,"The Kingsway,Montgomery Road,Old Mill North",M8X,43.653654,-79.506944
99,Church and Wellesley,M4Y,43.665860,-79.383160
100,Business Reply Mail Processing Centre 969 Eastern,M7Y,43.662744,-79.321558
101,"Humber Bay,King's Mill Park,Kingsway Park Sout...",M8Y,43.636258,-79.498509


#########question3

###open connection with 4square

In [54]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         238 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0        conda-forge
    geopy:         1.20.0-py_0      conda-forge

The following pac

In [55]:
CLIENT_ID = '43X1WLFVVAOIBBJWWT2MP4WHPHARAGBEUVR3DKJ1TS4ZS35F' # your Foursquare ID
CLIENT_SECRET = 'M30L3ASI2XPQRDGY31WPKYAKYSZP3VVRAKINQWKFSGLX4J2U' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 43X1WLFVVAOIBBJWWT2MP4WHPHARAGBEUVR3DKJ1TS4ZS35F
CLIENT_SECRET:M30L3ASI2XPQRDGY31WPKYAKYSZP3VVRAKINQWKFSGLX4J2U


#####get data for toronto

In [62]:
neighborhood_latitude = 43.6595255

neighborhood_longitude = -79.340923

LIMIT = 100 # limit of number of venues returned by Foursquare API



radius = 500 # define radius




url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=43X1WLFVVAOIBBJWWT2MP4WHPHARAGBEUVR3DKJ1TS4ZS35F&client_secret=M30L3ASI2XPQRDGY31WPKYAKYSZP3VVRAKINQWKFSGLX4J2U&v=20180605&ll=43.6595255,-79.340923&radius=500&limit=100'

In [63]:
toronto = requests.get(url).json()


In [65]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [66]:
venues = toronto['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Ed's Real Scoop,Ice Cream Shop,43.660656,-79.342019
1,Leslieville Pumps,Sandwich Place,43.660892,-79.340626
2,Queen Books,Bookstore,43.660651,-79.342267
3,Te Aro,Coffee Shop,43.661373,-79.338577
4,Hooked,Fish Market,43.660407,-79.343257


In [67]:
tor_onehot = pd.get_dummies(toronto['Venue Category'], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tor_onehot['Neighborhood'] = toronto['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tor_onehot.columns[-1]] + list(tor_onehot.columns[:-1])
tor_onehot = torn_onehot[fixed_columns]

tor_onehot.head()

KeyError: 'Venue Category'